In [30]:
import PyPDF2
import os
import glob
from io import StringIO
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser
import numpy as np

def convert_pdf_to_string(file_path):

    output_string = StringIO()
    with open(file_path, 'rb') as in_file:
        parser = PDFParser(in_file)
        doc = PDFDocument(parser)
        rsrcmgr = PDFResourceManager()
        device = TextConverter(rsrcmgr, output_string, laparams=LAParams())
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        for page in PDFPage.create_pages(doc):
            interpreter.process_page(page)

    return(output_string.getvalue())


class ProgBar:

    def __init__(self, n_elements,int_str):
        
        import sys

        self.n_elements = n_elements
        self.progress = 0

        print(int_str)

        # initiallizing progress bar

        info = '{:.2f}% - {:d} of {:d}'.format(0,0,n_elements)

        formated_bar = ' '*int(50)

        sys.stdout.write("\r")

        sys.stdout.write('[%s] %s' % (formated_bar,info))

        sys.stdout.flush()

    def update(self,prog_info=None):
        
        import sys

        if prog_info == None:

            self.progress += 1

            percent = (self.progress)/self.n_elements * 100 / 2

            info = '{:.2f}% - {:d} of {:d}'.format(percent*2,self.progress,self.n_elements)

            formated_bar = '-'* int (percent) + ' '*int(50-percent)

            sys.stdout.write("\r")

            sys.stdout.write('[%s] %s' % (formated_bar,info))

            sys.stdout.flush()


        else:

            self.progress += 1

            percent = (self.progress)/self.n_elements * 100 / 2

            info = '{:.2f}% - {:d} of {:d} '.format(percent*2,self.progress,self.n_elements) + prog_info

            formated_bar = '-'* int (percent) + ' '*int(50-percent)

            sys.stdout.write("\r")

            sys.stdout.write('[%s] %s' % (formated_bar,info))

            sys.stdout.flush()


def add_line(folder,line):

    # Open the file in append & read mode ('a+')

    with open(folder+"/conv.txt", "a+") as file_object:

        # Move read cursor to the start of file.
        file_object.seek(0)

        # If file is not empty then append '\n'
        data = file_object.read(100)

        if len(data) > 0 :
            file_object.write("\n")
            
        # Append text at the end of file
        file_object.write(line)



In [6]:
pages_struct = {}

folder = 'Capitulo_5'

reader = PyPDF2.PdfFileReader(folder + '/slide.pdf')

pdf_n_pages = reader.numPages

writer = PyPDF2.PdfFileWriter()

### Reading text and saving in the dictionary

# Initializing progress bar

bar = ProgBar(pdf_n_pages,'Reading text and saving in the dictionary...')


for pg in range(pdf_n_pages):

    pages_struct[pg] = {
        'figures': [],
        'text': 0,
        'equations': []
    }

    writer = PyPDF2.PdfFileWriter()

    output_filename = 'transition.pdf'

    page = reader.getPage(pg)

    writer.addPage(page)

    with open(output_filename, 'wb') as output:
        writer.write(output)

    del writer

    text = convert_pdf_to_string('transition.pdf')

    text = text.replace('\x0c',' ')
    text = text.replace('%','\%')
    text = text.split('\n')
    text = text[5:]

    pages_struct[pg]['text'] = text

    bar.update()


# Reading equations from txt

eq_file = folder + '/equations.txt'

with open(eq_file, 'r') as input:
        equations = input.read()


# formating strings 

equations = equations.replace('\\','')

equations = equations.split('\n')


# saving formated equations on the dictionary

# listing equation's figures

figures_list  = glob.glob(folder+'/equations/*')

# sorting list o files

page_list = []

# getting list of pages with equations

for i in range(len(figures_list)): 

    # extracting the page and format of the file's names

    name,form = figures_list[i].split('/')[-1].split('.')
    
    pg,_= name.split('-')

    # appending pages with eaqutions

    page_list.append(int(pg))

# sorting list of pages 

page_list = list(np.sort(page_list))

# creating dictionary for appending the list  of figures in each page

eq_dict = {}

for pg in page_list:

    eq_dict[pg] = []


# getting list of eqautions in each page

for i in range(len(figures_list)): 

    # extracting the page and format of the file's names

    name,form = figures_list[i].split('/')[-1].split('.')
    pg,num = name.split('-')

    # appending numbe of the equations in this page

    eq_dict[int(pg)].append(int(num))

    # sorting equations

    eq_dict[int(pg)] = list(np.sort(eq_dict[int(pg)]))

# setting count of equations

k = 1

for pg in eq_dict:

    for  num in eq_dict[pg]:

        # extracting the page and number from dictionary

        name = '-'.join((str(pg),str(num)))

        # writing LaTex syntax

        figure_call = [
            '\\begin{figure}[h!]',
            '\centering',
            ('\includegraphics[scale=0.5]{Pictures/' + folder + '/' + name + '.png}'),
            ('\caption{Equação' + str(k) + '}'),
            ('\label{fig:eq' + str(k) + '}'),
            '\end{figure}',
            ' ',
            ' '
        ]

        # incremeanting count

        k += 1

        # appending formated text in the dicctionary

        for i in range(len(figure_call)): 
            pages_struct[int(pg)-1]['equations'].append(figure_call[i])




# Reading figures and saving in the dictionary

figures_list  = glob.glob(folder+'/figures/*')

for i in range(len(figures_list)): 

    pg,ext = figures_list[i].split('/')[-1].split('.')

    figure_call = [
        '\\begin{figure}[h!]',
        '\centering',
        ('\includegraphics[scale=0.5]{Pictures/' + folder + '/' + pg + '.' + ext + '}'),
        '\caption{Caption}',
        '\label{fig:my_label}',
        '\end{figure}'
    ]

    for i in range(len(figure_call)): pages_struct[int(pg)-1]['figures'].append(figure_call[i])

Reading text and saving in the dictionary...
[--------------------------------------------------] 100.00% - 48 of 48

ValueError: invalid literal for int() with base 10: 'latexImage_58038d373106eb9e4988e36b21befe4b'

In [47]:
# listing equation's figures

figures_list  = glob.glob(folder+'/equations/*')

# sorting list o files

page_list = []

# getting list of pages with equations

for i in range(len(figures_list)): 

    # extracting the page and format of the file's names

    name,form = figures_list[i].split('/')[-1].split('.')
    
    pg,_= name.split('-')

    # appending pages with eaqutions

    page_list.append(int(pg))

# sorting list of pages 

page_list = list(np.sort(page_list))

# creating dictionary for appending the list  of figures in each page

eq_dict = {}

for pg in page_list:

    eq_dict[pg] = []


# getting list of eqautions in each page

for i in range(len(figures_list)): 

    # extracting the page and format of the file's names

    name,form = figures_list[i].split('/')[-1].split('.')
    pg,num = name.split('-')

    # appending numbe of the equations in this page

    eq_dict[int(pg)].append(int(num))

    # sorting equations

    eq_dict[int(pg)] = list(np.sort(eq_dict[int(pg)]))

# setting count of equations

k = 1

for pg in eq_dict:

    for  num in eq_dict[pg]:

        # extracting the page and number from dictionary

        name = '-'.join((str(pg),str(num)))

        # writing LaTex syntax

        figure_call = [
            '\\begin{figure}[h!]',
            '\centering',
            ('\includegraphics[scale=0.5]{Pictures/' + folder + '/' + name + '.png}'),
            ('\caption{Equação' + str(k) + '}'),
            ('\label{fig:eq' + str(k) + '}'),
            '\end{figure}',
            ' ',
            ' '
        ]

        # incremeanting count

        k += 1

        # appending formated text in the dicctionary

        for i in range(len(figure_call)): 
            pages_struct[int(pg)-1]['equations'].append(figure_call[i])



In [ ]:

# getting list of eqautions in each page

for i in range(len(figures_list)): 

    # extracting the page and format of the file's names

    name,form = figures_list[i].split('/')[-1].split('.')
    pg,num = name.split('-')

    # appending numbe of the equations in this page

    eq_dict[int(pg)].append(int(num))

    # sorting equations

    eq_dict[int(pg)] = list(np.sort(eq_dict[int(pg)]))

# setting count of equations

k = 1

for pg in range(1,max(page_list)+1):

    for  num in eq_dict[pg]:

        # extracting the page and number from dictionary

        name = '-'.join((str(pg),str(num)))

        # writing LaTex syntax

        figure_call = [
            '\\begin{figure}[h!]',
            '\centering',
            ('\includegraphics[scale=0.5]{Pictures/' + folder + '/' + name + '.png}'),
            ('\caption{Equação' + str(k) + '}'),
            ('\label{fig:eq' + str(k) + '}'),
            '\end{figure}'
        ]

        # incremeanting count

        k += 1

        # appending formated text in the dicctionary

        for i in range(len(figure_call)): 
            pages_struct[int(pg)-1]['equations'].append(figure_call[i])

